<a href="https://colab.research.google.com/github/drustantmetar/DataFrame1/blob/main/Spark_DataFrame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

spark data frame
**bold text**

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [7]:
spark = SparkSession\
.builder\
.appName("Spark-SQL")\
.master("local")\
.getOrCreate()
